In [3]:
# Tests whether or not the optimzations to the fractal generation algorithm actually did anything

import time

import ctypes
import numpy as np
from numpy.ctypeslib import ndpointer
from PIL import Image

lib = ctypes.CDLL(r"C:\Users\aiden\Fractals\optimized_fractals_dll\build\Release\optimized_fractals.dll")

lib.standard_fractal.argtypes = [
    ctypes.c_float, ctypes.c_float, ctypes.c_float,
    ctypes.c_int, ctypes.c_int, ctypes.c_int,
    ctypes.POINTER(ctypes.c_ubyte),
    ctypes.POINTER(ctypes.c_float)
]

lib.slow_fractal.argtypes = [
    ctypes.c_float, ctypes.c_float, ctypes.c_float,
    ctypes.c_int, ctypes.c_int, ctypes.c_int,
    ctypes.POINTER(ctypes.c_ubyte),
    ctypes.POINTER(ctypes.c_float)
]

lib.get_last_error.restype = ctypes.c_char_p

In [2]:
width, height = 1920, 1080

coeffs = [  1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
            0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,
            0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
            0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
            0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
            0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
            0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
coeffArray = ctypes.c_float * 52
coeffs_c = coeffArray(*coeffs)
buffer = np.zeros(width * height * 3, dtype=np.uint8)

err = lib.slow_fractal(
    -0.75, 0.0, 0.002,
    width, height, 1000,
    buffer.ctypes.data_as(ctypes.POINTER(ctypes.c_ubyte)),
    coeffs_c
)
if err == -1:
    print(lib.get_last_error())

img = Image.frombuffer("RGB", (width, height), buffer, "raw", "RGB", 0, 1)
img.save("mandelbrot.png")

In [ ]:

width, height = 1920, 1080

coeffs = [  1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
            0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,
            0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
            0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
            0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
            0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
            0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
coeffArray = ctypes.c_float * 52
coeffs_c = coeffArray(*coeffs)
buffer = np.zeros(width * height * 3, dtype=np.uint8)

err = lib.standard_fractal(
    -0.75, 0.0, 0.002,
    width, height, 1000,
    buffer.ctypes.data_as(ctypes.POINTER(ctypes.c_ubyte)),
    coeffs_c
)
if err == -1:
    print(lib.get_last_error())

img = Image.frombuffer("RGB", (width, height), buffer, "raw", "RGB", 0, 1)
img.save("mandelbrot.png")

standard_time = time.perf_counter()
for i in range(100):
    err = lib.standard_fractal(
        -0.75, 0.0, 0.002,
        width, height, 1000,
        buffer.ctypes.data_as(ctypes.POINTER(ctypes.c_ubyte)),
        coeffs_c
    )
    if err == -1:
        print(lib.get_last_error())

standard_time = time.perf_counter() - standard_time

slow_time = time.perf_counter() 
for i in range(100):
    err = lib.slow_fractal(
        -0.75, 0.0, 0.002,
        width, height, 1000,
        buffer.ctypes.data_as(ctypes.POINTER(ctypes.c_ubyte)),
        coeffs_c
    )
    if err == -1:
        print(lib.get_last_error())

slow_time = time.perf_counter() - slow_time

# MASSIVE speed increase by optimizing kernel code!! 15x faster!!
print(standard_time)
print(slow_time)

0.6544648999697529
10.380970700003672


In [7]:
standard_time / 100

0.006544648999697529